# 1. Vision Transformer介绍
![vit](pngs/vit_architecture.png)
Transformer模型成为自然语言处理(NLP)领域的事实标准后，许多人都尝试将Transformer结构应用到计算机视觉(CV)领域。这其中最出名的，第一次将Transformer应用在大规模图像识别上的便是谷歌提出的vision Transformer,简称VIT。VIT将一副h*w的图像看成是N个大小为s*s的小网格，$N=(h*w)/(s*s)$. 每个小网格称为一个patch,每个patch可以映射为一个一维的向量来表示。然后将N个patch展开，就将图像转化为了类似自然语言序列的数据格式，可以直接使用Transformer来处理。

# 2. Vision Transformer中的核心概念
### 1. 如何将图像作为Transformer的输入
Transformer是一种序列模型，用于处理序列数据，而彩色图像对应一个RGB三维像素矩阵，那么如何才能将图像转化为Transformer模型的输入呢?     
不难想到，最简单的方法就是将像素矩阵全部展开，展开成1维的序列，但是这样会导致序列长度爆炸，比如对于一个192*192大小的灰度图像，展开后我们将得到一个长为36864的序列，再经过嵌入之后，存储空间和计算成本都将爆炸式增长，显然是不可接受的。    
为了降低输入序列的长度，有两种方案，一种是尽量降低输入图像的分辨率，比如可以resize为小分辨率的图像，或者先使用CNN进行特征提取，使用降采样后的特征图作为输入而不是直接使用原始的高分辨率图片作为输入。另一种方案是不再把每一个像素点单独作为一个token,而是把一个n*n大小的窗口内的所有像素当成一个token,Vision Transformer中即采用了这样的方法。
### 2. 位置编码
与处理自然语言的Transformer一样，VIT同样需要对图像patch进行位置编码，位置编码可以反映patch之间的彼此临近关系。回想一下，Transformer中使用了人工设计的变周期的三角函数来进行位置编码，而VIT中的位置编码不是人工设计的，而是创建了一个可训练的参数，将位置编码交给网络自己去学习。
### 3. 特殊的分类token
Transformer的编码器的输出的形状与编码器的输入的形状完全一致，都是类似(batch_size, seq_length, d_model)，要进行图像识别，还需要再拼接上全连接层。  

如果直接将编码器的输出作为全连接层的输入，会有两个问题,一是全连接层需要输入的维度固定，不同分辨率的图像提取出的patch数量不一样，即seq_length数量不一样，为了满足这一限制，必须将所有图像resize成一样的，这样就不能发挥出Transformer架构本身处理不定长序列的能力，第二个问题是Transformer模型的嵌入层一般都很大，所以seq_length*d_model也将会非常大，导致全连接层的运算量极大，影响训练和推理时的性能。
    
 为了解决这两个问题，Vision Transformer借鉴了BERT模型中的做法，引入了一个特殊的分类token, 经过编码器后，认为这个特殊的分类token中已经编码了图像的所有信息，分类时，只需要将这一个token对应的向量输入全连接层进行回归即可。这样对于任意长度的序列，都可以保证全连接层的输入是固定的，而且极大地降低了全连接层的输入维度。
具体实现中，所谓的分类token,就是在编码器的输入的seq_length维度的0号位置concat了一个维度也为d_model的向量，将输入从(batch_size, seq_length, d_model)变成了(batch_size, 1 + seq_length, d_model)

# 3. Vision Transformer的实现
VIT只使用了Transformer的编码器部分，然后在编码器上拼接了一个MLP来做回归，负责最后的识别。编码器部分的实现和标准Transformer中的实现完全相同，不再赘述。

需要注意的是位置编码(PositionalEncoding)的实现，提取图像patch(ImagePatcher)的实现和cls_token的表示。

In [ ]:
import math
import torch
from torch import nn


class VisionTransformer(nn.Module):
    def __init__(self,
                 patcher,
                 d_model,
                 attention_head,
                 stack_number,
                 dff,
                 max_length,
                 class_number):
        super().__init__()
        self.patcher = patcher
        self.pe = PositionalEncoding(d_model, max_length)
        self.cls_token = nn.Parameter(torch.randn(1, 1, d_model), requires_grad=True) # 特殊的分类token
        self.encoder = Encoder(stack_number, attention_head, d_model, dff)
        self.head = HeadLayer(d_model, class_number, d_model)
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x):
        x = self.pe(self.patcher(x))
        cls_token = self.cls_token.expand(x.shape[0], -1, -1) # expand to match the batch dim
        x = torch.cat([cls_token, x], dim=1)
        x = self.encoder(x)
        x = x[:, 0, :]
        x = self.norm(x)
        output = self.head(x)
        return output


# 将图像转化为patch的序列，并进行线性映射
class ImagePatcher(nn.Module):
    def __init__(self, ic, oc, kernel_size, stride):
        super().__init__()
        self.conv = nn.Conv2d(ic, oc, kernel_size, stride)

    def forward(self, x):
        x = self.conv(x) # 注意此时的维度顺序为batch_size, channel, h, w
        batch_size, channel, h, w = x.shape
        x = x.view(batch_size, channel, -1)
        x = x.permute(0, 2, 1) # batch_size, seq_length, d_model
        return x


# 可训练的位置编码
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_length):
        super().__init__()
        pe = torch.zeros(1, max_length, d_model)
        pe = nn.Parameter(pe, requires_grad=True)
        self.pe = pe

    def forward(self, x):
        x = x + self.pe[:, :x.shape[1], :]
        return x


# 前馈神经网络
class FeedForwardNetwork(nn.Module):
    def __init__(self, ic, hidden, dropout=None):
        super().__init__()
        self.linear1 = nn.Linear(ic, hidden)
        self.linear2 = nn.Linear(hidden, ic)
        self.dropout = nn.Dropout(0.1) if dropout is not None else None
        self.act = nn.ReLU()

    def forward(self, x):
        x = self.act(self.linear1(x))
        if self.dropout is not None:
            x = self.dropout(x)
        x = self.linear2(x)
        return x


# 分类头。两层全连接中间加个relu
class HeadLayer(nn.Module):
    def __init__(self, ic, class_number, hidden):
        super().__init__()
        self.linear1 = nn.Linear(ic, hidden)
        self.act = nn.ReLU()
        self.linear2 = nn.Linear(hidden, class_number)

    def forward(self, x):
        x = self.linear2(self.act(self.linear1(x)))
        return x


# 多头自注意力
# 这段代码的实现也是抄的现成的。奇文共欣赏，疑义相与析
class MultiHeadAttention(nn.Module):
    def __init__(self, head_number, d_model):
        """
        :param head_number: 自注意力头的数量
        :param d_model: 隐藏层的维度
        """
        super().__init__()
        self.h = head_number
        self.d_model = d_model
        self.dk = d_model // head_number
        self.q_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.output = nn.Linear(d_model, d_model)
        self.softmax = nn.Softmax(-1)
        self.dropout = nn.Dropout(0.1)

    def head_split(self, tensor, batch_size):
        # 将(batch_size, seq_len, d_model) reshape成 (batch_size, seq_len, h, d_model//h)
        # 然后再转置第1和第2个维度，变成(batch_size, h, seq_len, d_model/h)
        return tensor.view(batch_size, -1, self.h, self.dk).transpose(1, 2)

    def head_concat(self, similarity, batch_szie):
        # 恢复计算注意力之前的形状
        return similarity.transpose(1, 2).contiguous() \
            .view(batch_szie, -1, self.d_model)

    def cal_attention(self, q, k, v, mask=None):
        """
        论文中的公式 Attention(K,Q,V) = softmax(Q@(K^T)/dk**0.5)@V
        ^T 表示矩阵转置
        @ 表示矩阵乘法
        """
        similarity = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.dk)
        if mask is not None:
            mask = mask.unsqueeze(1)
            # 将mask为0的位置填充为绝对值非常大的负数
            # 这样经过softmax后，其对应的权重就会非常接近0, 从而起到掩码的效果
            similarity = similarity.masked_fill(mask == 0, -1e9)
        similarity = self.softmax(similarity)
        similarity = self.dropout(similarity)

        output = torch.matmul(similarity, v)
        return output

    def forward(self, q, k, v, mask=None):
        """
        q,k,v即自注意力公式中的Q,K,V，mask表示掩码
        """
        batch_size, seq_length, d = q.size()
        q = self.q_linear(q)
        k = self.k_linear(k)
        v = self.v_linear(v)
        # 分成多个头
        q = self.head_split(q, batch_size)
        k = self.head_split(k, batch_size)
        v = self.head_split(v, batch_size)
        similarity = self.cal_attention(q, k, v, mask)
        # 合并多个头的结果
        similarity = self.head_concat(similarity, batch_size)

        # 再使用一个线性层， 投影一次
        output = self.output(similarity)
        return output


# 编码器层。
# 每个编码器层由两个sublayer组成，即一个多头注意力层和一个前馈网络
class EncoderLayer(nn.Module):
    def __init__(self, head_number, d_model, d_ff, dropout=0.1):
        super().__init__()

        # mha
        self.mha = MultiHeadAttention(head_number, d_model)
        self.norm1 = nn.LayerNorm(d_model)

        # mlp
        self.mlp = FeedForwardNetwork(d_model, d_ff)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        x2 = self.norm1(x)

        y = self.dropout1(self.mha(x2, x2, x2, mask))
        # 注意残差连接是和norm之前的输入相加，norm之后的不在一个数量级
        y = y + x

        y2 = self.norm2(y)
        y2 = self.dropout2(self.mlp(y2))
        y2 = y + y2

        return y2


# 编码器部分
# 编码器就是N个编码器层堆叠起来。论文中为6个编码器层
class Encoder(nn.Module):
    def __init__(self, stack=6, multi_head=8, d_model=512, d_ff=2048):
        """
        :param stack: 堆叠多少个编码器层
        :param multi_head: 多头注意力头的数量
        :param d_model: 隐藏层的维度
        """
        super().__init__()
        self.encoder_stack = []
        for i in range(stack):
            encoder_layer = EncoderLayer(multi_head, d_model, d_ff)
            self.encoder_stack.append(encoder_layer)
        self.encoder = nn.ModuleList(self.encoder_stack)
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x, mask=None):
        for encoder_layer in self.encoder:
            x = encoder_layer(x, mask)
        x = self.norm(x)
        return x


def build_model(img_channel, patch_size, d_model, attention_head, stack_number, dff, max_length, class_number):
    """
    :param img_channel: 输入的图像的通道数，对于RGB图像，通道数为3，对于灰度图，通道数为1
    :param patch_size: 每个图像patch的大小，比如(2,2)
    :param d_model: 嵌入维度
    :param attention_head: 多头自注意力的头数
    :param stack_number: 自注意力block的数量
    :param dff: 前馈神经网络隐藏层的规模
    :param max_length: 最大的序列长度，用于创建位置编码
    :param class_number: 分类的类别数量
    :return: VIT 模型
    """
    patcher = ImagePatcher(img_channel, d_model, patch_size, patch_size)
    model = VisionTransformer(patcher, d_model, attention_head, stack_number, dff, max_length, class_number)
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return model


# 简单测试
def basic_test():
    d_model = 256
    attention_head = 4
    stack_number = 4
    dff = 512
    max_length = 200
    class_number = 10
    model = build_model(1, (2,2), d_model, attention_head, stack_number, dff, max_length, class_number)
    print("Model Info")
    print(model)

    sample = torch.randn(1, 1, 28, 28)
    output = model(sample)
    print("output shape: {}".format(output.shape))

basic_test()

# 4. 在MNIST数据集上训练VIT
接下来，我们将在MNIST数据集上训练一个小型的Vision Transformer模型，VIT模型有很多的参数可以用来控制模型的规模，包括嵌入层的维度，自注意力block的数量，多头自注意力的数量等，可以根据需求灵活调整，此处仅在MNIST数据集上做演示，所以构建了一个较小的Vision Transformer模型。

In [ ]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

batch_size = 64
device = 'cuda'
epochs = 30
train_dataset = torchvision.datasets.MNIST(root='data',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)
train_data = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


vit_model = build_model(1, (2,2), 128, 4, 4, 1024, 300, 10)
vit_model = vit_model.to(device)
optimizer = torch.optim.Adam(vit_model.parameters(), lr=1e-4, betas=(0.9, 0.98), eps=1e-9)
loss_func = F.cross_entropy

for epoch in range(1, epochs+1):
    loss_sum = 0
    acc_sum = 0
    n = 0
    for step, batch_data in enumerate(train_data):
        x, label = batch_data
        x = x.to(device)
        label = label.to(device)
        output = vit_model(x)
        loss = loss_func(output, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        pred_index = F.softmax(output, dim=1)
        pred_index = torch.argmax(pred_index, dim=1)
        acc = torch.sum(pred_index == label) / len(label)
        loss_sum += loss
        acc_sum += acc
        n += 1

    loss_avg = loss_sum / n
    acc_avg = acc_sum / n
    print("Epoch:{}  Average Loss:{:.3f}  Accuracy:{:.3f}".format(epoch, loss_avg, acc_avg))

# 5. 总结
你已经成功实现并训练了一个Vision Transformer模型，尝试在你自己的数据集上训练并优化它吧。
如果你还想继续学习关于Vision Transformer模型的知识，以下是一些可供参考的学习资料
+ [Vision Transformer论文精读](https://www.bilibili.com/video/BV15P4y137jb/?spm_id_from=333.999.0.0&vd_source=7ba4ab07bcd248758aff19a21fc5010b)
+ [Vision Transformer论文原文](https://arxiv.org/abs/2010.11929)